In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

## FGNet-LOPO -- Hierarchical Model

In [210]:
df_fgnet = pd.read_csv("data/FGNet-LOPO.csv")
df_fgnet["ageclass"] = df_fgnet.age.apply(
    lambda r: 0 if r < 18 else 1
).astype(int)

df_fgnet['ageclass_ext'] = pd.cut(
    df_fgnet.age,
    bins=[0,13,16,20,24,28,32,36,40,100],
    labels=[0,1,2,3,4,5,6,7,8],
    include_lowest=True,
    ordered=False,
)

print(df_fgnet.columns)
print(f' <18: {df_fgnet[df_fgnet.age < 18].shape[0]}\n>=18: {df_fgnet[df_fgnet.age >= 18].shape[0]}')
print(df_fgnet[["ageclass"]].value_counts())
print(df_fgnet[["ageclass_ext"]].value_counts())
df_fgnet[["age", "ageclass", "ageclass_ext"]].sample(20)

Index(['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10',
       ...
       'b105', 'b106', 'b107', 'b108', 'b109', 'age', 'ID', 'Gender_0M_1F',
       'ageclass', 'ageclass_ext'],
      dtype='object', length=114)
 <18: 640
>=18: 362
ageclass
0           640
1           362
dtype: int64
ageclass_ext
0               513
2               118
1                99
3                64
8                60
4                51
5                38
6                36
7                23
dtype: int64


,age,ageclass,ageclass_ext
677,10,0,0
797,30,1,5
960,9,0,0
321,19,1,2
356,33,1,6
352,22,1,3
313,4,0,0
483,52,1,8
871,13,0,0
271,25,1,4


In [211]:
def df_shape_table(*df_dicts):
    title_str = f"{'DataFrame':>15} | {'Shape':15}"
    print(title_str)
    for df_dict in df_dicts:
        print(f"{''.join(['-'] * 25):^33}")
        for name, df in df_dict.items():
            print(f"{name:>15} | { str(df.shape) :<15}")

In [229]:
df_yng = df_fgnet.iloc[df_fgnet[df_fgnet.ageclass == 0].index, :]
df_old = df_fgnet.iloc[df_fgnet[df_fgnet.ageclass == 1].index, :]

X = df_fgnet.drop(["age", "ID", "Gender_0M_1F", "ageclass", "ageclass_ext"], axis=1)
X_yng = df_yng.drop(["age", "ID", "Gender_0M_1F", "ageclass", "ageclass_ext"], axis=1)
X_old = df_old.drop(["age", "ID", "Gender_0M_1F", "ageclass", "ageclass_ext"], axis=1)

y = df_fgnet[["ID", "age", "ageclass", "ageclass_ext", "Gender_0M_1F"]]
y_yng = df_yng[["ID", "age", "ageclass", "ageclass_ext", "Gender_0M_1F"]]
y_old = df_old[["ID", "age", "ageclass", "ageclass_ext", "Gender_0M_1F"]]

df_shape_table({'X': X, 'X Young': X_yng, 'X Old': X_old},
               {'y': y, 'y Young': y_yng, 'y Old': y_old})

      DataFrame | Shape          
    -------------------------    
              X | (1002, 109)    
        X Young | (640, 109)     
          X Old | (362, 109)     
    -------------------------    
              y | (1002, 5)      
        y Young | (640, 5)       
          y Old | (362, 5)       


In [230]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
X_yng_train, X_yng_test, y_yng_train, y_yng_test = train_test_split(X_yng, y_yng, test_size=.2)
X_old_train, X_old_test, y_old_train, y_old_test = train_test_split(X_old, y_old, test_size=.2)

df_shape_table(
    {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test},
    {'X_yng_train': X_yng_train, 'X_yng_test': X_yng_test, 'y_yng_train': y_yng_train, 'y_yng_test': y_yng_test},
    {'X_old_train': X_old_train, 'X_old_test': X_old_test, 'y_old_train': y_old_train, 'y_old_test': y_old_test}
)

      DataFrame | Shape          
    -------------------------    
        X_train | (801, 109)     
         X_test | (201, 109)     
        y_train | (801, 5)       
         y_test | (201, 5)       
    -------------------------    
    X_yng_train | (512, 109)     
     X_yng_test | (128, 109)     
    y_yng_train | (512, 5)       
     y_yng_test | (128, 5)       
    -------------------------    
    X_old_train | (289, 109)     
     X_old_test | (73, 109)      
    y_old_train | (289, 5)       
     y_old_test | (73, 5)        


In [231]:
model_metrics = {}

for i in range(10):
    model_metrics[i] = {
        "clf_true": [],
        "clf_pred": [],
        "reg_true": [],
        "reg_pred": [],
        "total": 0,
        "score": 0,
        "error": 0.
    }
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
    X_yng_train, X_yng_test, y_yng_train, y_yng_test = train_test_split(X_yng, y_yng, test_size=.2)
    X_old_train, X_old_test, y_old_train, y_old_test = train_test_split(X_old, y_old, test_size=.2)
    
    svm = SVC(C=1000, gamma=.01, kernel='rbf').fit(X_train, y_train["ageclass"])
    reg_yng = Ridge(alpha=.1).fit(X_yng_train, y_yng_train["age"])
    reg_old = Ridge(alpha=.05).fit(X_old_train, y_old_train["age"])
    
    for idx in X_test.index:
        row = X_test.loc[idx].values.reshape(1, -1)
        
        age_true = y_test.loc[idx, "age"]
        ageclass_true = y_test.loc[idx, "ageclass"]
        ageclass_pred = svm.predict(row)
        
        if ageclass_pred not in [0, 1]:
            raise ValueError("ageclass must be either 0 or 1")
            
        age_pred = reg_yng.predict(row) if ageclass_pred == 0 else reg_old.predict(row)
        
        model_metrics[i]["error"] += abs(age_pred - age_true)
        model_metrics[i]["score"] += 1 if np.round(age_pred) == age_true else 0
        model_metrics[i]["total"] += 1
        model_metrics[i]["clf_true"].append(ageclass_true)
        model_metrics[i]["clf_pred"].append(ageclass_pred)
        model_metrics[i]["reg_true"].append(age_true)
        model_metrics[i]["reg_pred"].append(age_pred)
        
    print(model_metrics[i]["error"] / model_metrics[i]["total"])

[3.53497685]
[3.39084203]
[2.77399149]
[3.54276065]
[3.64512329]
[4.23237177]
[3.78589167]
[4.33483476]
[3.84990299]
[3.84852682]


### Classification/Regression Metrics

In [236]:
from sklearn import metrics as m

def calculate_metrics(model_metrics, multi_class=False):
    clf_metrics = {
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F1 Score': [],
        'Jaccard': [],
    }

    reg_metrics = {
        "MAE": [],
        "MSE": [],
        "RMSE": [],
        "R2": [],
        "MAPE": [],
        "Median AE": [],
        "Max Error": [],
    }

    for _, metrics in model_metrics.items():
        clf_true, clf_pred = metrics["clf_true"], metrics["clf_pred"]
        reg_true, reg_pred = metrics["reg_true"], metrics["reg_pred"]
        
        if not multi_class: 
            clf_metrics["Accuracy"].append(m.accuracy_score(clf_true, clf_pred))
            clf_metrics["Precision"].append(m.precision_score(clf_true, clf_pred))
            clf_metrics["Recall"].append(m.recall_score(clf_true, clf_pred))
            clf_metrics["F1 Score"].append(m.f1_score(clf_true, clf_pred))
            clf_metrics["Jaccard"].append(m.jaccard_score(clf_true, clf_pred))
            
        else:
            clf_metrics["Accuracy"].append(m.accuracy_score(clf_true, clf_pred))
            clf_metrics["Precision"].append(m.precision_score(clf_true, clf_pred, average=None))
            clf_metrics["Recall"].append(m.recall_score(clf_true, clf_pred, average=None))
            clf_metrics["F1 Score"].append(m.f1_score(clf_true, clf_pred, average=None))
            clf_metrics["Jaccard"].append(m.jaccard_score(clf_true, clf_pred, average=None))
        
        reg_metrics["MAE"].append(m.mean_absolute_error(reg_true, reg_pred))
        reg_metrics["MSE"].append(m.mean_squared_error(reg_true, reg_pred))
        reg_metrics["RMSE"].append(m.mean_squared_error(reg_true, reg_pred, squared=False))
        reg_metrics["R2"].append(m.r2_score(reg_true, reg_pred))
        reg_metrics["MAPE"].append(m.mean_absolute_percentage_error(reg_true, reg_pred))
        reg_metrics["Median AE"].append(m.median_absolute_error(reg_true, reg_pred))
        reg_metrics["Max Error"].append(m.max_error(reg_true, reg_pred))
            
        
            
    clf_metrics = pd.DataFrame(clf_metrics).round(4)
    reg_metrics = pd.DataFrame(reg_metrics).round(4)
    
    return pd.concat({'Regression': reg_metrics, 'Classification': clf_metrics}, axis=1)


calculate_metrics(model_metrics)

Regression                                                             \
         MAE      MSE    RMSE      R2          MAPE Median AE Max Error   
0     3.5350  29.5460  5.4356  0.7953  1.292912e+14    2.0037   25.4272   
1     3.3908  24.8408  4.9840  0.8463  2.187681e+14    1.9130   19.1298   
2     2.7740  18.9841  4.3571  0.8533  2.169543e+14    1.7245   24.8856   
3     3.5428  31.2232  5.5878  0.8045  4.735621e+14    1.8586   26.3321   
4     3.6451  30.9643  5.5646  0.8236  9.965972e+13    2.2153   26.0097   
5     4.2324  44.8570  6.6975  0.7585  5.574749e+14    2.2667   39.7041   
6     3.7859  34.6006  5.8822  0.7412  3.102499e+14    1.9515   22.9888   
7     4.3348  44.3319  6.6582  0.7109  3.202494e+14    2.3471   28.9183   
8     3.8499  38.2005  6.1807  0.7887  3.262207e+14    2.0209   27.6731   
9     3.8485  46.4463  6.8152  0.7244  3.687514e+14    1.8822   43.1244   

  Classification                                     
        Accuracy Precision  Recall F1 Score Jaccard  
0         0.8657    0.8072  0.8590   0.8323  0.7128  
1         0.8905    0.8857  0.8158   0.8493  0.7381  
2         0.8806    0.8305  0.7778   0.8033  0.6712  
3         0.8607    0.7534  0.8462   0.7971  0.6627  
4         0.8756    0.8118  0.8846   0.8466  0.7340  
5         0.8408    0.8261  0.7403   0.7808  0.6404  
6         0.8557    0.7606  0.8182   0.7883  0.6506  
7         0.8607    0.7738  0.8784   0.8228  0.6989  
8         0.8706    0.7791  0.9054   0.8375  0.7204  
9         0.8507    0.7467  0.8358   0.7887  0.6512

## FGNet-LOPO -- Hierarchical Model w/ LOPOCV

In [215]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR

In [238]:
model_metrics = {}

for i in range(10):
    model_metrics[i] = {
        "clf_true": [],
        "clf_pred": [],
        "reg_true": [],
        "reg_pred": [],
        "total": 0,
        "score": 0,
        "error": 0.
    }
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
    X_yng_train, _, y_yng_train, _ = train_test_split(X_yng, y_yng, test_size=.2)
    X_old_train, _, y_old_train, _ = train_test_split(X_old, y_old, test_size=.2)
    
    svm = SVC(C=1000, gamma = .01, kernel='rbf').fit(X_train, y_train["ageclass"])
    reg_yng = Ridge(alpha=.1).fit(X_yng_train, y_yng_train["age"])
    reg_old = make_pipeline(StandardScaler(), 
                            LinearSVR(epsilon=2.97, tol=1e-6, C=14.1)
                            ).fit(X_old_train, y_old_train["age"])
    
    for idx in X_test.index:
        row = X_test.loc[idx].values.reshape(1, -1)
        
        age_true = y_test.loc[idx, "age"]
        ageclass_true = y_test.loc[idx, "ageclass"]
        ageclass_pred = svm.predict(row)
        
        if ageclass_pred not in [0, 1]:
            raise ValueError("ageclass must be either 0 or 1")
            
        age_pred = reg_yng.predict(row) if ageclass_pred == 0 else reg_old.predict(row)
        
        model_metrics[i]["error"] += abs(age_pred - age_true)
        model_metrics[i]["score"] += 1 if np.round(age_pred) == age_true else 0
        model_metrics[i]["total"] += 1
        model_metrics[i]["clf_true"].append(ageclass_true)
        model_metrics[i]["clf_pred"].append(ageclass_pred)
        model_metrics[i]["reg_true"].append(age_true)
        model_metrics[i]["reg_pred"].append(age_pred)
        
    print(model_metrics[i]["error"] / model_metrics[i]["total"])
    
calculate_metrics(model_metrics)

/Users/Michael/.pyenv/versions/anaconda3-2021.05/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/Michael/.pyenv/versions/anaconda3-2021.05/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[3.65545948]
[4.32428187]


/Users/Michael/.pyenv/versions/anaconda3-2021.05/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/Michael/.pyenv/versions/anaconda3-2021.05/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[3.87951727]
[3.76746533]


/Users/Michael/.pyenv/versions/anaconda3-2021.05/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/Michael/.pyenv/versions/anaconda3-2021.05/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


...

/Users/Michael/.pyenv/versions/anaconda3-2021.05/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/Michael/.pyenv/versions/anaconda3-2021.05/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[3.66390468]
[3.63325925]


/Users/Michael/.pyenv/versions/anaconda3-2021.05/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/Michael/.pyenv/versions/anaconda3-2021.05/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[3.7733828]
[3.65001095]


Regression                                                             \
         MAE      MSE    RMSE      R2          MAPE Median AE Max Error   
0     3.6555  39.8964  6.3164  0.7723  3.351796e+14    2.1577   43.9484   
1     4.3243  56.9823  7.5487  0.7303  4.399835e+14    2.5123   56.5344   
2     3.8795  46.6493  6.8300  0.7272  3.717301e+14    2.2966   45.0624   
3     3.7675  35.9083  5.9924  0.7732  2.971551e+14    2.2556   26.0294   
4     3.6178  32.7586  5.7235  0.8114  6.258522e+14    2.2422   26.4104   
5     4.0348  49.8506  7.0605  0.6716  1.449055e+15    2.4573   52.9916   
6     3.6639  30.9487  5.5632  0.8184  3.138326e+14    2.4798   26.8061   
7     3.6333  34.7237  5.8927  0.8302  2.676836e+14    2.1080   26.9174   
8     3.7734  35.8303  5.9858  0.7765  2.162036e+14    2.2364   29.5687   
9     3.6500  32.8605  5.7324  0.7443  4.000776e+14    2.4600   27.4062   

  Classification                                     
        Accuracy Precision  Recall F1 Score Jaccard  
0         0.8856    0.9028  0.8025   0.8497  0.7386  
1         0.8756    0.8500  0.8395   0.8447  0.7312  
2         0.8756    0.8472  0.8133   0.8299  0.7093  
3         0.8706    0.8118  0.8734   0.8415  0.7263  
4         0.9055    0.8971  0.8356   0.8652  0.7625  
5         0.8308    0.7500  0.7714   0.7606  0.6136  
6         0.8408    0.7625  0.8243   0.7922  0.6559  
7         0.8657    0.7895  0.8451   0.8163  0.6897  
8         0.8706    0.8356  0.8133   0.8243  0.7011  
9         0.9005    0.8065  0.8621   0.8333  0.7143

## Multilevel One-Hot

In [239]:
y['ageclass_ext'].unique()

[0, 1, 2, 3, 4, 5, 6, 7, 8]
Categories (9, int64): [0, 1, 2, 3, ..., 5, 6, 7, 8]

In [241]:
model_metrics = {}

for i in range(10):
    model_metrics[i] = {
        "clf_true": [],
        "clf_pred": [],
        "reg_true": [],
        "reg_pred": [],
        "total": 0,
        "score": 0,
        "error": 0.
    }
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
    
    svm = SVC(C=1000, gamma=.01, kernel='rbf').fit(X_train, y_train['ageclass_ext'])
    reg = [Ridge(alpha=.1),
           make_pipeline(StandardScaler(),
                         LinearSVR(epsilon=2.97, tol=1e-6, C=14.1)),
           Ridge(alpha=.1), Ridge(alpha=.1), Ridge(alpha=.1),
           Ridge(alpha=.1), Ridge(alpha=.1), Ridge(alpha=.1),
           Ridge(alpha=.1)]
    
    for r in range(len(reg)):
        indices = y_train[y_train['ageclass_ext'] == r].index
        reg[r].fit(X_train.loc[indices], y_train.loc[indices,'age'])
        
    # reg_yng = Ridge(alpha=.1).fit(X_yng_train, y_yng_train["age"])
    # reg_old = make_pipeline(StandardScaler(), 
    #                         LinearSVR(epsilon=2.97, tol=1e-6, C=14.1)
    #                         ).fit(X_old_train, y_old_train["age"])
    
    for idx in X_test.index:
        row = X_test.loc[idx].values.reshape(1, -1)
        ageclass_true = y_test.loc[idx, 'ageclass_ext']
        ageclass_pred = svm.predict(row)
        
        # if ageclass_pred not in list(y['ageclass_ext'].unique()):
        #     raise ValueError("ageclass must be in {}".format(str(y['ageclass_ext'].unique())))
        
        age_pred = reg[ageclass_pred[0]].predict(row)
        # age_pred = reg_yng.predict(row) if ageclass_pred == 0 else reg_old.predict(row)
        
        model_metrics[i]["error"] += abs(age_pred - age_true)
        model_metrics[i]["score"] += 1 if np.round(age_pred) == age_true else 0
        model_metrics[i]["total"] += 1
        model_metrics[i]["clf_true"].append(ageclass_true)
        model_metrics[i]["clf_pred"].append(ageclass_pred)
        model_metrics[i]["reg_true"].append(age_true)
        model_metrics[i]["reg_pred"].append(age_pred)
        
    print(model_metrics[i]["error"] / model_metrics[i]["total"])
    
calculate_metrics(model_metrics, multi_class=True)

[22.40487333]
[22.3637456]
[21.91954686]
[21.55971455]
[22.41040527]
[20.87370454]
[21.25529178]
[21.99564676]
[21.80617786]
[21.98663905]


/Users/Michael/.pyenv/versions/anaconda3-2021.05/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Regression                                                      \
         MAE       MSE     RMSE   R2    MAPE Median AE Max Error   
0    22.4049  580.1298  24.0859  0.0  0.6401   24.5055   37.7476   
1    22.3637  572.5872  23.9288  0.0  0.6390   24.8422   38.0741   
2    21.9195  559.8045  23.6602  0.0  0.6263   24.3559   36.2682   
3    21.5597  541.7902  23.2764  0.0  0.6160   24.5118   37.9202   
4    22.4104  575.0551  23.9803  0.0  0.6403   24.5400   36.7575   
5    20.8737  515.1087  22.6960  0.0  0.5964   23.7080   38.9869   
6    21.2553  529.6033  23.0131  0.0  0.6073   22.8874   39.5593   
7    21.9956  564.7382  23.7642  0.0  0.6284   24.2402   37.9878   
8    21.8062  557.4183  23.6097  0.0  0.6230   24.2289   36.8489   
9    21.9866  564.6140  23.7616  0.0  0.6282   24.2347   39.4899   

  Classification                                                     \
        Accuracy                                          Precision   
0         0.5423  [0.8240740740740741, 0.125, 0.1923076923076923...   
1         0.5970  [0.8319327731092437, 0.3333333333333333, 0.333...   
2         0.5721  [0.8571428571428571, 0.16666666666666666, 0.33...   
3         0.5473  [0.8240740740740741, 0.22727272727272727, 0.23...   
4         0.6020  [0.8305084745762712, 0.35714285714285715, 0.39...   
5         0.5721  [0.8640776699029126, 0.2727272727272727, 0.344...   
6         0.5572  [0.8155339805825242, 0.38095238095238093, 0.31...   
7         0.5821  [0.8648648648648649, 0.25, 0.3448275862068966,...   
8         0.5970  [0.8407079646017699, 0.4666666666666667, 0.285...   
9         0.5373  [0.8108108108108109, 0.21052631578947367, 0.21...   

                                                      \
                                              Recall   
0  [0.898989898989899, 0.21428571428571427, 0.192...   
1  [0.9339622641509434, 0.2727272727272727, 0.25,...   
2  [0.9056603773584906, 0.125, 0.5294117647058824...   
3  [0.8725490196078431, 0.22727272727272727, 0.26...   
4  [0.9333333333333333, 0.21739130434782608, 0.40...   
5  [0.89, 0.15789473684210525, 0.4761904761904761...   
6  [0.8842105263157894, 0.3076923076923077, 0.375...   
7  [0.897196261682243, 0.23809523809523808, 0.454...   
8  [0.9313725490196079, 0.35, 0.3157894736842105,...   
9  [0.8737864077669902, 0.2, 0.2692307692307692, ...   

                                                      \
                                            F1 Score   
0  [0.859903381642512, 0.15789473684210525, 0.192...   
1  [0.88, 0.3, 0.28571428571428575, 0.33333333333...   
2  [0.8807339449541285, 0.14285714285714288, 0.40...   
3  [0.8476190476190476, 0.22727272727272727, 0.24...   
4  [0.8789237668161435, 0.27027027027027023, 0.39...   
5  [0.8768472906403941, 0.19999999999999998, 0.39...   
6  [0.8484848484848484, 0.3404255319148936, 0.339...   
7  [0.8807339449541284, 0.24390243902439024, 0.39...   
8  [0.883720930232558, 0.4, 0.3, 0.23999999999999...   
9  [0.8411214953271028, 0.20512820512820512, 0.24...   

                                                      
                                             Jaccard  
0  [0.7542372881355932, 0.08571428571428572, 0.10...  
1  [0.7857142857142857, 0.17647058823529413, 0.16...  
2  [0.7868852459016393, 0.07692307692307693, 0.25...  
3  [0.7355371900826446, 0.1282051282051282, 0.139...  
4  [0.784, 0.15625, 0.25, 0.05555555555555555, 0....  
5  [0.7807017543859649, 0.1111111111111111, 0.25,...  
6  [0.7368421052631579, 0.20512820512820512, 0.20...  
7  [0.7868852459016393, 0.1388888888888889, 0.243...  
8  [0.7916666666666666, 0.25, 0.17647058823529413...  
9  [0.7258064516129032, 0.11428571428571428, 0.13...

## LOPOCV

In [247]:
import importlib
import pyfgnet.crossval as cv
importlib.reload(cv)

<module 'pyfgnet.crossval' from '/Users/Michael/Documents/work/2021-UNCW-SVSM/face-recognition/pyfgnet/crossval.py'>

In [253]:
model_metrics = {}

# for i in range(10):
#     model_metrics[i] = {
#         "clf_true": [],
#         "clf_pred": [],
#         "reg_true": [],
#         "reg_pred": [],
#         "total": 0,
#         "score": 0,
#         "error": 0.
#     }
    
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
    # X_yng_train, X_yng_test, y_yng_train, y_yng_test = train_test_split(X_yng, y_yng, test_size=.2)
    # X_old_train, X_old_test, y_old_train, y_old_test = train_test_split(X_old, y_old, test_size=.2)
    
for i in y.ID.unique():
    model_metrics[i] = {
        "clf_true": [],
        "clf_pred": [],
        "reg_true": [],
        "reg_pred": [],
        "total": 0,
        "score": 0,
        "error": 0.
    }
    
for train, valid in cv.LOPOCV(shuffle=True).split(X, y):
    X_train, X_valid = X.loc[train], X.loc[valid]
    y_train, y_valid = y.loc[train], y.loc[valid]
    yng, old = y_train[y_train["ageclass"] == 0].index, y_train[y_train["ageclass"] == 1].index
    
    svm = SVC(C=1000, gamma=.01, kernel='rbf').fit(X_train, y_train["ageclass"])
    reg_yng = Ridge(alpha=.1).fit(X_train.loc[yng], y_train.loc[yng, "age"])
    reg_old = Ridge(alpha=.05).fit(X_train.loc[old], y_train.loc[old, "age"])
    
    for idx in X_valid.index:
        row = X_valid.loc[idx].values.reshape(1, -1)
        
        age_true = y_valid.loc[idx, "age"]
        ageclass_true = y_valid.loc[idx, "ageclass"]
        ageclass_pred = svm.predict(row)
        
        if ageclass_pred not in [0, 1]:
            raise ValueError("ageclass must be either 0 or 1")
            
        age_pred = reg_yng.predict(row) if ageclass_pred == 0 else reg_old.predict(row)
        
        uid = y_valid.loc[idx, "ID"]
        model_metrics[uid]["error"] += abs(age_pred - age_true)
        model_metrics[uid]["score"] += 1 if np.round(age_pred) == age_true else 0
        model_metrics[uid]["total"] += 1
        model_metrics[uid]["clf_true"].append(ageclass_true)
        model_metrics[uid]["clf_pred"].append(ageclass_pred)
        model_metrics[uid]["reg_true"].append(age_true)
        model_metrics[uid]["reg_pred"].append(age_pred)
        
calculate_metrics(model_metrics)

[5.55031048]
[2.32028257]
[5.38395261]
[2.45059377]
[4.70467101]
[3.37019087]
[4.93541329]
[6.41960006]
[2.53647378]
[5.92109395]
[5.50569784]
[6.53543424]
[13.92291929]
[6.56822698]
[4.77080099]
[1.45480666]
[1.68476561]
[2.66440466]
[1.50567331]
[2.09739641]
[0.94226099]
[3.05782216]
[5.10811991]
[6.46463509]
[3.33001117]
[6.13792965]
[7.71453206]
[6.08674987]
[2.78067757]
[5.09837332]
[2.43685159]
[5.38135122]
[6.21806969]
[2.72089249]
[6.34753936]
[3.55304003]
[3.17943093]
[9.41581349]
[2.71738442]
[9.26890886]
[8.06379573]
[4.30350372]
[1.17870624]
[9.5300144]
[6.50881052]
[0.88892811]
[8.30066499]
[5.22316975]
[2.22691556]
[8.39304018]
[5.57385444]
[5.18599667]
[4.13933843]
[3.110968]
[4.5695712]
[4.2706061]
[1.53329754]
[14.98428124]
[1.08007578]
[8.9632405]
[1.49311332]
[3.70358046]
[1.96366548]
[9.32755383]
[9.59885718]
[3.45355881]
[5.23145155]
[6.63134152]
[9.25490175]
[5.45408953]
[5.57151814]
[3.13896458]
[2.89999306]
[4.6915157]
[3.94588244]
[4.29112412]
[1.63614791]
[2.0